In [1]:
import os, uuid, json, requests
from datetime import datetime, timedelta, timezone
from pymongo import MongoClient
from dotenv import load_dotenv
from bson import json_util as bsonju

load_dotenv()

PREVIOUS_DOC = {
  "id": "9f3a27e6-1b52-4e0b-bc6e-62c1f2a7c8b9",
  "previous_form_qr_code": "20251011",
  "generated_by": "Ella Reyes",
  "generator_employee_number": "50325",
  "current_form_qr_code": "20251012",
  "date": "2025-10-12",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "created": { "$date": "2025-10-12T08:00:00Z" },
  "items": [
    {"item_number":"001","id":"5d6b2f7d-9a5b-4d7f-9f38-2a4b9f0b1c01","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000101","type":"fuel","name":"Premium Gasoline","price":69.5,"unit":"liter","previous_quantity":520,"addstock":300,"sold":250,"current_quantity":570},
    {"item_number":"002","id":"7c8f3e2b-6a41-46b2-9d9b-2b98c1b7f202","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000102","type":"fuel","name":"Regular Gasoline","price":64.75,"unit":"liter","previous_quantity":680,"addstock":400,"sold":320,"current_quantity":760},
    {"item_number":"003","id":"a3c0c1de-2c3e-4c90-9f0b-6e2d9f8a9303","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000103","type":"fuel","name":"Premium Diesel","price":61.2,"unit":"liter","previous_quantity":450,"addstock":350,"sold":280,"current_quantity":520},
    {"item_number":"004","id":"c92c7d1e-2b7f-4c6d-9d1a-8f2a3b5c4d04","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000201","type":"lubricant","name":"Engine Oil SAE 40","price":250,"unit":"liter","previous_quantity":60,"addstock":40,"sold":25,"current_quantity":75},
    {"item_number":"005","id":"f1a2b3c4-d5e6-47f8-9a0b-b1c2d3e4f505","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000202","type":"lubricant","name":"Automatic Transmission Fluid (ATF)","price":310,"unit":"liter","previous_quantity":50,"addstock":30,"sold":20,"current_quantity":60},
    {"item_number":"006","id":"0b1c2d3e-4f56-47a8-9b0c-1d2e3f4a5b06","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000203","type":"lubricant","name":"Gear Oil EP 90","price":285,"unit":"liter","previous_quantity":45,"addstock":25,"sold":15,"current_quantity":55},
    {"item_number":"007","id":"1c2d3e4f-5a6b-4c7d-8e9f-0a1b2c3d4e07","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000204","type":"lubricant","name":"Hydraulic Oil ISO 68","price":295,"unit":"liter","previous_quantity":35,"addstock":25,"sold":10,"current_quantity":50},
    {"item_number":"008","id":"2d3e4f5a-6b7c-4d8e-9f0a-1b2c3d4e5f08","created":"2025-10-12T08:00:00Z","updated":"2025-10-12T08:00:00Z","barcode":"4809992000205","type":"lubricant","name":"2T Motorcycle Oil","price":180,"unit":"liter","previous_quantity":80,"addstock":60,"sold":45,"current_quantity":95}
  ],
  "cashier": "Liam Santos",
  "cashier_employee_number": "50321",
  "recorder": "Ella Reyes",
  "recorder_employee_number": "50325",
  "is_empty": False
}

QR_API_URL = os.getenv(
    "GENERATE_QR_CODE_URL",
    "https://faas-sgp1-18bc02ac.doserverless.co/api/v1/web/fn-86217a9f-1135-4904-b49a-fe070d4e10c7/forms/generate-qr-code"
)

def _next_business_date(s):
    d = datetime.strptime(s, "%Y-%m-%d")
    return (d + timedelta(days=1)).strftime("%Y-%m-%d")

def _now_utc():
    return datetime.now(timezone.utc)

def _fetch_new_qr():
    try:
        r = requests.get(QR_API_URL, timeout=10)
        r.raise_for_status()
        data = r.json()
        val = data.get("current_form_qr_code")
        if not val:
            raise ValueError("QR API response missing current_form_qr_code")
        return str(val)
    except Exception:
        return uuid.uuid4().hex[:8]

def _roll_items(prev_items):
    now = _now_utc()
    rolled = []
    for idx, it in enumerate(prev_items, start=1):
        rolled.append({
            "item_number": f"{idx:03d}",
            "id": str(uuid.uuid4()),
            "created": now,
            "updated": now,
            "barcode": it["barcode"],
            "type": it["type"],
            "name": it["name"],
            "price": it["price"],
            "unit": it["unit"],
            "previous_quantity": it.get("current_quantity", 0),
            "addstock": 0,
            "sold": 0,
            "current_quantity": 0
        })
    return rolled

def build_new_inventory(prev_doc):
    now = _now_utc()
    return {
        "id": str(uuid.uuid4()),
        "previous_form_qr_code": prev_doc["current_form_qr_code"],
        "current_form_qr_code": _fetch_new_qr(),
        "date": _next_business_date(prev_doc["date"]),
        "location": prev_doc["location"],
        "tin": prev_doc["tin"],
        "branch": prev_doc["branch"],
        "generated_by": prev_doc.get("recorder") or prev_doc.get("generated_by"),
        "generator_employee_number": prev_doc.get("recorder_employee_number") or prev_doc.get("generator_employee_number"),
        "created": now,
        "items": _roll_items(prev_doc.get("items", [])),
        "is_empty": True
    }

new_doc = build_new_inventory(PREVIOUS_DOC)

uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")
collection_name = "inventories"

if uri and database:
    client = MongoClient(uri)
    coll = client[database][collection_name]
    result = coll.insert_one(new_doc)
    print(f"inserted_id: {result.inserted_id}")

print(bsonju.dumps(new_doc, ensure_ascii=False, indent=2))


inserted_id: 68f14a85e85501f19104b642
{
  "id": "956598bb-5d3d-47de-87d7-136a46020529",
  "previous_form_qr_code": "20251012",
  "current_form_qr_code": "42214212",
  "date": "2025-10-13",
  "location": "JEF Gas Station – Sikatuna Branch",
  "tin": "123-456-789",
  "branch": "JEF Biosciences Fuel Division",
  "generated_by": "Ella Reyes",
  "generator_employee_number": "50325",
  "created": {
    "$date": 1760643716313
  },
  "items": [
    {
      "item_number": "001",
      "id": "0522679f-d1b3-4497-84ac-ca7f833ea080",
      "created": {
        "$date": 1760643717626
      },
      "updated": {
        "$date": 1760643717626
      },
      "barcode": "4809992000101",
      "type": "fuel",
      "name": "Premium Gasoline",
      "price": 69.5,
      "unit": "liter",
      "previous_quantity": 570,
      "addstock": 0,
      "sold": 0,
      "current_quantity": 0
    },
    {
      "item_number": "002",
      "id": "fb8e82d7-f1b7-4169-a4ab-9ac3ad3b9a6b",
      "created": {
        "$d